In [1]:
%pip install torch unsloth transformers datasets trl tqdm

  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.24.0+cu128
    Uninstalling torchvision-0.24.0+cu128:
      Successfully uninstalled torchvision-0.24.0+cu128
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuda-python 12.9.5 requires cuda-bindings~=12.9.5, but you have cuda-bindings 12.9.4 which is incompatible.
torchaudio 2.9.0+cu128 requires torch==2.9.0, but you have torch 2.10.0 which is incompatible.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatible.


In [2]:
import sys
import os
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

from src.config import SYSTEM_INSTRUCTION, MAX_SEQ_LENGTH
from src.data_utils import process, prompt
from src.model_utils import load_model
from src.inference import extract_graph

/tmp/ipython-input-3547963855.py:6: UserWarning: WARNING: Unsloth should be imported before [trl, transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


ModuleNotFoundError: No module named 'src'

In [ ]:
raw_data = process(prompt=SYSTEM_INSTRUCTION, amount=20000)
dataset = Dataset.from_list(raw_data)
print(f"✅ Data processed. {len(dataset)} examples ready.")

In [ ]:
model, tokenizer = load_model()
print("✅ Model loaded and adapters attached.")

In [ ]:
dataset = dataset.map(lambda x: prompt(x, tokenizer), batched=True)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

print("Starting Training")
trainer.train()
print("Done!")